# Algorithms for Data Science

## Matching Bids to Queries

### 1. Preliminaries 

The objective of this lab is to implement the Adwords algorithm in the particular case when we have $N$ advertisers having budget $B$ and each bidding an amount of $1$. Moreover, queries arrive in batches of $B$ queries, $N$ times, and only advertises $A_i,\dots,A_n$ bid on round $i$. The optimal algorithm would have revenue $NB$. Our aim is to see what the revenue of the BALANCE algorithm is.

In [12]:
import random
import numpy as np
import math
#parameters
N = 10
B = 10
 

### 2. BALANCE algorithm

The balance algorithm simply gives the query to the bidder having the most budget available. In this case, BALANCE achieves a competitive ratio of $1-1/e$.

In [2]:
# array of budgets
Bs = [B]*N
revenue = 0

for i in range(N): #rounds
  for j in range(B): #queries
    idx = np.argmax(Bs[i:]) # taking the index with the most budget
    #print (Bs[i:])
    if Bs[i+idx]>0:
      Bs[i+idx] -= 1
      revenue += 1
      print (f'Round {i} query {j} -- allocated to bidder {i+idx}')
    else:
      print (f'Round {i} query {j} -- not allocated')

print (f'Total revenue: {revenue}')

Round 0 query 0 -- allocated to bidder 0
Round 0 query 1 -- allocated to bidder 1
Round 0 query 2 -- allocated to bidder 2
Round 0 query 3 -- allocated to bidder 3
Round 0 query 4 -- allocated to bidder 4
Round 0 query 5 -- allocated to bidder 5
Round 0 query 6 -- allocated to bidder 6
Round 0 query 7 -- allocated to bidder 7
Round 0 query 8 -- allocated to bidder 8
Round 0 query 9 -- allocated to bidder 9
Round 1 query 0 -- allocated to bidder 1
Round 1 query 1 -- allocated to bidder 2
Round 1 query 2 -- allocated to bidder 3
Round 1 query 3 -- allocated to bidder 4
Round 1 query 4 -- allocated to bidder 5
Round 1 query 5 -- allocated to bidder 6
Round 1 query 6 -- allocated to bidder 7
Round 1 query 7 -- allocated to bidder 8
Round 1 query 8 -- allocated to bidder 9
Round 1 query 9 -- allocated to bidder 1
Round 2 query 0 -- allocated to bidder 2
Round 2 query 1 -- allocated to bidder 3
Round 2 query 2 -- allocated to bidder 4
Round 2 query 3 -- allocated to bidder 5
Round 2 query 4 

### 3. **TASK** Generalized BALANCE Algorithm

Implement the generalized BALANCE algorithm, where the bids can be arbitrary, and the bidder is selected based on the $\psi$ function presented in the lecture.

In [79]:
# YOUR CODE HERE

# let's identify the query by a number / id

# prices of bids are between 2 and 20
# number of possible bids for each bidder is between 3 and 30
class bidder:
    def __init__(self, id_bidder, bid_price, budget):
        self.id_bidder = id_bidder
        self.bid_price = bid_price
        self.budget = budget
        self.spent = 0
        self.target_queries = []
        
    def add_target_query(self,query_id):
        if query_id not in self.target_queries:
            self.target_queries.append(query_id)
            
    def is_bidder_intrested_in_query(self, query):
        return query in self.target_queries
    
    def is_bidding_active(self):
        return (self.budget>self.spent)
    
    def attribute_query(self):
        self.spent += self.bid_price
        
    def fi_value(self):
        return self.bid_price*(1-math.exp(1-(self.spent/self.budget)))
    
    def __str__(self):
        return "bidder "+self.id_bidder.__str__()+" bidding:"+self.bid_price.__str__()+" spent : "+self.spent.__str__()+"/"+self.budget.__str__()+" : "+self.target_queries.__str__()
    
class bidding_manager:
    def __init__(self, nb_bidders, nb_queries, intrest_per_bidder):
        self.nb_bidders = nb_bidders
        self.intrest_per_bidder = intrest_per_bidder
        self.bidders = [] # list of bidder_objects
        self.nb_queries = nb_queries
        self.available = range(nb_queries)
        
        self.stream_history = []
        
    
    def randomize_the_demande(self):
        for i in range(self.nb_bidders):
            price = random.randrange(18)+2
            stock = random.randrange(27)+3
            bid = bidder(i,price,stock*price)
            for j in range(self.intrest_per_bidder):
                bid.add_target_query(random.randrange(self.nb_queries))
            print(bid.__str__())
            self.bidders.append(bid)
        
    
    def select_best_bid(self, call):
        possible_values = []
        possible_bidders = []
        
        for bid in self.bidders:
            if ( bid.is_bidding_active() and bid.is_bidder_intrested_in_query(call)):
                possible_values.append(bid.fi_value())
                possible_bidders.append(bid)
        
        #choose the bid with max values
        if(len(possible_values) > 0):
            idx = np.argmax(possible_values)
            print("selected bidder id:%d"%possible_bidders[idx].id_bidder)
            possible_bidders[idx].attribute_query()
            print(possible_bidders[idx].__str__())
        else:
            print("bidders all inactive")
        
        
    
    def simulate_random_queries(self, stream_size):
        for i in range(stream_size):  
            call = random.randrange(self.nb_queries)
            print(" the query for %d:"%call)
            self.stream_history.append(call)
            self.select_best_bid(call)
            
    def get_total_gains(self):
        total = 0
        for bid in self.bidders:
            total += bid.spent
            
        print("TOTAL GAINS IS %d"%total)
        return total
    
            

_You can use this cell to write your discussion of the results_

In [78]:
mn = bidding_manager(10,10,3)
mn.randomize_the_demande()

mn.simulate_random_queries(150)
mn.get_total_gains()

bidder 0 bidding:13 spent : 0/78 : [7, 4, 5]
bidder 1 bidding:3 spent : 0/21 : [2, 0, 3]
bidder 2 bidding:9 spent : 0/126 : [8, 9, 0]
bidder 3 bidding:6 spent : 0/84 : [4, 2]
bidder 4 bidding:16 spent : 0/304 : [0, 6]
bidder 5 bidding:14 spent : 0/308 : [5, 0, 8]
bidder 6 bidding:4 spent : 0/84 : [8, 1]
bidder 7 bidding:11 spent : 0/66 : [5, 8, 4]
bidder 8 bidding:15 spent : 0/120 : [2, 3, 4]
bidder 9 bidding:12 spent : 0/108 : [9, 5, 3]
 the query for 7:
selected bidder id:0
bidder 0 bidding:13 spent : 13/78 : [7, 4, 5]
 the query for 5:
selected bidder id:0
bidder 0 bidding:13 spent : 26/78 : [7, 4, 5]
 the query for 1:
selected bidder id:6
bidder 6 bidding:4 spent : 4/84 : [8, 1]
 the query for 0:
selected bidder id:1
bidder 1 bidding:3 spent : 3/21 : [2, 0, 3]
 the query for 2:
selected bidder id:1
bidder 1 bidding:3 spent : 6/21 : [2, 0, 3]
 the query for 0:
selected bidder id:1
bidder 1 bidding:3 spent : 9/21 : [2, 0, 3]
 the query for 1:
selected bidder id:6
bidder 6 bidding:4 s

In [83]:
from statistics import mean,median

try_ = 10
data = []

for i in range(try_):
    mn = bidding_manager(10,10,3)
    mn.randomize_the_demande()

    mn.simulate_random_queries(150)
    data.append(mn.get_total_gains())
    

print(" average gains is : %d"%mean(data))
print(" min gains is : %d"%min(data))   
print(" max gains is : %d"%max(data))   
    
    
    

bidder 0 bidding:19 spent : 0/76 : [1, 5, 0]
bidder 1 bidding:6 spent : 0/174 : [7, 4]
bidder 2 bidding:4 spent : 0/92 : [7, 5]
bidder 3 bidding:13 spent : 0/260 : [7, 3, 5]
bidder 4 bidding:15 spent : 0/150 : [4, 5]
bidder 5 bidding:8 spent : 0/176 : [5, 9, 2]
bidder 6 bidding:9 spent : 0/99 : [0, 2, 5]
bidder 7 bidding:13 spent : 0/104 : [9, 8, 5]
bidder 8 bidding:8 spent : 0/40 : [9, 7, 8]
bidder 9 bidding:4 spent : 0/16 : [8, 0, 5]
 the query for 6:
bidders all inactive
 the query for 7:
selected bidder id:2
bidder 2 bidding:4 spent : 4/92 : [7, 5]
 the query for 1:
selected bidder id:0
bidder 0 bidding:19 spent : 19/76 : [1, 5, 0]
 the query for 3:
selected bidder id:3
bidder 3 bidding:13 spent : 13/260 : [7, 3, 5]
 the query for 7:
selected bidder id:2
bidder 2 bidding:4 spent : 8/92 : [7, 5]
 the query for 6:
bidders all inactive
 the query for 9:
selected bidder id:5
bidder 5 bidding:8 spent : 8/176 : [5, 9, 2]
 the query for 1:
selected bidder id:0
bidder 0 bidding:19 spent : 

bidders all inactive
 the query for 7:
selected bidder id:8
bidder 8 bidding:14 spent : 294/406 : [4, 2, 7]
 the query for 9:
bidders all inactive
 the query for 2:
selected bidder id:9
bidder 9 bidding:10 spent : 180/190 : [0, 8, 2]
TOTAL GAINS IS 1353
bidder 0 bidding:18 spent : 0/270 : [3, 4, 5]
bidder 1 bidding:3 spent : 0/39 : [6, 7, 1]
bidder 2 bidding:6 spent : 0/24 : [1, 7]
bidder 3 bidding:10 spent : 0/250 : [9, 5]
bidder 4 bidding:17 spent : 0/170 : [9, 8]
bidder 5 bidding:7 spent : 0/161 : [5, 1, 9]
bidder 6 bidding:14 spent : 0/238 : [8, 9]
bidder 7 bidding:15 spent : 0/240 : [5, 1, 0]
bidder 8 bidding:6 spent : 0/18 : [1, 8, 3]
bidder 9 bidding:2 spent : 0/56 : [5, 6, 9]
 the query for 5:
selected bidder id:9
bidder 9 bidding:2 spent : 2/56 : [5, 6, 9]
 the query for 0:
selected bidder id:7
bidder 7 bidding:15 spent : 15/240 : [5, 1, 0]
 the query for 8:
selected bidder id:8
bidder 8 bidding:6 spent : 6/18 : [1, 8, 3]
 the query for 4:
selected bidder id:0
bidder 0 bidding

selected bidder id:4
bidder 4 bidding:2 spent : 2/16 : [4, 5, 3]
 the query for 8:
selected bidder id:3
bidder 3 bidding:9 spent : 9/117 : [9, 7, 8]
 the query for 3:
selected bidder id:4
bidder 4 bidding:2 spent : 4/16 : [4, 5, 3]
 the query for 9:
selected bidder id:7
bidder 7 bidding:2 spent : 2/32 : [5, 9, 1]
 the query for 4:
selected bidder id:4
bidder 4 bidding:2 spent : 6/16 : [4, 5, 3]
 the query for 1:
selected bidder id:7
bidder 7 bidding:2 spent : 4/32 : [5, 9, 1]
 the query for 8:
selected bidder id:3
bidder 3 bidding:9 spent : 18/117 : [9, 7, 8]
 the query for 9:
selected bidder id:7
bidder 7 bidding:2 spent : 6/32 : [5, 9, 1]
 the query for 5:
selected bidder id:4
bidder 4 bidding:2 spent : 8/16 : [4, 5, 3]
 the query for 2:
selected bidder id:6
bidder 6 bidding:3 spent : 3/33 : [2, 3]
 the query for 9:
selected bidder id:7
bidder 7 bidding:2 spent : 8/32 : [5, 9, 1]
 the query for 3:
selected bidder id:4
bidder 4 bidding:2 spent : 10/16 : [4, 5, 3]
 the query for 6:
sel

-8